# Day 21
# Transfer Learning

- Goal:
    1. .

In [2]:
from tensorflow import keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras import backend as K
from tensorflow.keras.datasets import cifar10
from sklearn.preprocessing import OneHotEncoder
import numpy as np

input_tensor = Input(shape=(32, 32, 3))
model = keras.applications.Xception(include_top=False, weights='imagenet', input_tensor=input_tensor, classes=10)

x = model.output
predictions = Dense(10, activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print("Layers: ", len(model.layers))

Layers:  133


In [3]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

In [4]:
def normalize(x_train, x_test):
    mean = np.mean(x_train, axis=(0, 1, 2, 3))
    std = np.std(x_train, axis=(0, 1, 2, 3))
    x_train = (x_train - mean)/(std + 1e-7)
    x_test = (x_test - mean)/(std + 1e-7)
    return x_train, x_test

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = normalize(x_train, x_test)
one_hot = OneHotEncoder()
y_train = one_hot.fit_transform(y_train).toarray()
y_test = one_hot.transform(y_test).toarray()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train, y_train, batch_size=32, epochs=100)

C:\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


ValueError: A target array with shape (50000, 10) was passed for an output of shape (None, 1, 1, 10) while using as loss `categorical_crossentropy`. This loss expects targets to have the same shape as the output.